<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_08_Describe_Related_Images_from_Web_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to search the Web and ingest web pages, crawl image links, and describe the images. It also shows how to search the image descriptions based on the original Web search to find images related to your search text.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [1]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [2]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [3]:
from typing import List, Optional

# Create specification for OpenAI GPT-4o
async def create_openai_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="OpenAI GPT-4o",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=enums.OpenAIModels.GPT4O_128K,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create specification for Anthropic Sonnet 3.5
async def create_anthropic_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Anthropic Claude Sonnet 3.5",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=enums.AnthropicModels.CLAUDE_3_5_SONNET,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create enrichment workflow to crawl only image links from web pages, and enrich them with Groq Llava 1.5
async def create_workflow(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Link Crawling",
        extraction=input_types.ExtractionWorkflowStageInput(
            jobs=[
                input_types.ExtractionWorkflowJobInput(
                    connector=input_types.EntityExtractionConnectorInput(
                        type=enums.EntityExtractionServiceTypes.MODEL_IMAGE,
                        modelImage=input_types.ModelImageExtractionPropertiesInput(
                            specification=input_types.EntityReferenceInput(id=specification_id)
                        )
                    )
                )
            ]
        ),
        enrichment=input_types.EnrichmentWorkflowStageInput(
            link=input_types.LinkStrategyInput(
                enableCrawling=True,
                allowContentDomain=True,
                allowedLinks=[enums.LinkTypes.FILE],
                allowedFiles=[enums.FileTypes.IMAGE],
            )
        )
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_feed(search_text: str, read_limit: int, workflow_id: str):
    if graphlit.client is None:
        return;

    input = input_types.FeedInput(
        name="Web Search",
        type=enums.FeedTypes.SEARCH,
        search=input_types.SearchFeedPropertiesInput(
            text=search_text,
            readLimit=read_limit
        ),
        workflow=input_types.EntityReferenceInput(
            id=workflow_id
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

# Locate images ingested by feed
async def query_contents(feed_id: str, search: Optional[str] = None):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents(
            filter=input_types.ContentFilter(
                search=search,
                searchType=enums.SearchTypes.HYBRID,
                fileTypes=[enums.FileTypes.IMAGE],
                feeds=[
                    input_types.EntityReferenceFilter(
                        id=feed_id
                    )
                ]
            )
        )

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)


async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Execute Graphlit example

In [4]:
from IPython.display import display, Markdown, Image
import time

# Remove any existing feeds and workflows; only needed for notebook example
await delete_all_workflows()
await delete_all_feeds()

print('Deleted all feeds.')

read_limit = 1 # how many search results to ingest from feed

search_text = "Seattle Kraken mascot Buoy"

# NOTE: uncomment just one of these to use different models
specification_id = await create_anthropic_specification()
#specification_id = await create_openai_specification()

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        feed_id = await create_feed(search_text, read_limit, workflow_id)

        if feed_id is not None:
            print(f'Created feed [{feed_id}].')

            # Wait for feed to complete, since ingestion happens asychronously
            done = False
            time.sleep(5)
            while not done:
                done = await is_feed_done(feed_id)

                if not done:
                    time.sleep(2)

            print(f'Completed feed [{feed_id}].')

            # Show all the images crawled from the web page
            contents = await query_contents(feed_id)

            if contents is not None:
                print(f'Found {len(contents)} images in feed [{feed_id}].')
                print()

                for content in contents:
                    if content is not None and content.image is not None:
                        display(Image(url=content.image_uri, width=256))

            print()

            # Show just the images related to the search text, and the image text and descriptions
            contents = await query_contents(feed_id, search_text)

            if contents is not None:
                print(f'Found {len(contents)} images in feed [{feed_id}] related to [{search_text}].')
                print()

                for content in contents:
                    if content is not None:
                        display(Markdown(f'## {content.name} [{content.id}]:'))
                        display(Markdown(f'Original image size: {content.image.width}x{content.image.height}'))

                        if content.markdown is not None:
                            display(Markdown(f'Text in image: {content.markdown}'))

                        if content.image is not None:
                            display(Image(url=content.image_uri, width=512))

                            if content.image.description is not None:
                                display(Markdown(f'### Description:\n{content.image.description}'))

Deleted all feeds.
Created specification [2cb2713a-e71b-476f-be81-cdd2240d948f].
Created workflow [9ff18e57-9c1b-4989-b17a-423a283ead4c].
Created feed [19dc2b0d-4ae3-4998-841a-489272fb2ac1].
Completed feed [19dc2b0d-4ae3-4998-841a-489272fb2ac1].
Found 15 images in feed [19dc2b0d-4ae3-4998-841a-489272fb2ac1].




Found 1 images in feed [19dc2b0d-4ae3-4998-841a-489272fb2ac1] related to [Seattle Kraken mascot Buoy].



## Starbucks_vjdmhs.png [2637e3cc-93a3-410e-b57c-7f6745b69762]:

Original image size: 841x96

Text in image: 

### Description:
Blog Post

Blog Post

Blog Post